In [1]:
import pandas as pd
df_weather = pd.read_json(r"C:\Users\Linds\Repos\East_River\data\Processed\time_processed_weather.json")
df_weather.head()

,location,datetime,temperature,wind_speed,wind_degree,precip,humidity,visibility,pressure,cloudcover,heatindex,dewpoint,windchill,windgust,feelslike,uv_index,obs_tms_lcl,wind_dir,weather_descriptions
0,"Aberdeen, South Dakota",2020-12-31 06:00:00+00:00,12.0,7.0,197.0,0.0,90.0,6.0,30.120,69.0,12.0,9.0,0.0,15.0,0.0,1.0,2020-12-31T06:00:00.000Z,SSW,Cloudy
1,"Aberdeen, South Dakota",2020-12-31 06:30:00+00:00,11.0,7.0,196.0,0.0,90.0,6.0,30.105,61.0,11.0,9.0,0.0,15.5,0.0,1.0,2020-12-31T06:00:00.000Z,SSW,Cloudy
2,"Aberdeen, South Dakota",2020-12-31 07:00:00+00:00,10.0,7.0,195.0,0.0,90.0,6.0,30.090,53.0,10.0,9.0,0.0,16.0,0.0,1.0,2020-12-31T07:00:00.000Z,SSW,Partly cloudy
3,"Aberdeen, South Dakota",2020-12-31 07:30:00+00:00,10.0,7.0,194.0,0.0,89.5,6.0,30.090,45.5,10.0,9.0,0.0,16.0,0.0,1.0,2020-12-31T07:00:00.000Z,SSW,Partly cloudy
4,"Aberdeen, South Dakota",2020-12-31 08:00:00+00:00,10.0,7.0,193.0,0.0,89.0,6.0,30.090,38.0,10.0,9.0,0.0,16.0,0.0,1.0,2020-12-31T08:00:00.000Z,SSW,Partly cloudy


Extract unique locations

In [2]:
unique_locations = df_weather["location"].unique()
print(unique_locations)

['Aberdeen, South Dakota' 'Alsville, South Dakota' 'Ames, South Dakota'
 'Andover, South Dakota' 'Arco, Minnesota' 'Arlington Beach, South Dakota'
 'Armour, South Dakota' 'Astoria, South Dakota' 'Avon, South Dakota'
 'Beardsley, Minnesota' 'Beresford, South Dakota'
 'Big Springs, South Dakota' 'Bloomfield, South Dakota'
 'Blunt, South Dakota' 'Boisberg, Minnesota' 'Bonilla, South Dakota'
 'Bowdle, South Dakota' 'Bradley, South Dakota' 'Brandon, South Dakota'
 'Brandt, South Dakota' 'Brentford, South Dakota' 'Britton, South Dakota'
 'Brookings, South Dakota' 'Burkmere, South Dakota' 'Canby, Minnesota'
 'Canton, South Dakota' 'Carpenter, South Dakota'
 'Cedar Grove Colony, South Dakota' 'Chamberlain, South Dakota'
 'Chancellor, South Dakota' 'Charlesville, Minnesota'
 'Clark Colony, South Dakota' 'Clark, South Dakota'
 'Clayton, South Dakota' 'Clear Lake, South Dakota' 'Colman, South Dakota'
 'Colton, South Dakota' 'Country Estates Subdivision, South Dakota'
 'Craven Corner, South Dakota

In [3]:
print(len(unique_locations))

174


Geocode locations to match NOAA records

In [4]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="weather_app")

def geocode_location(location):
    try:
        geo = geolocator.geocode(location)
        if geo:
            return geo.latitude, geo.longitude
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
    return None, None

# Create a lookup for each unique location
location_coords = {}
for loc in unique_locations:
    lat, lon = geocode_location(loc)
    location_coords[loc] = (lat, lon)
    # Sleep a bit to avoid overwhelming the service
    time.sleep(1)


Add geocode to df_weather

In [5]:
df_weather['latitude'] = df_weather['location'].map(lambda loc: location_coords.get(loc, (None, None))[0])
df_weather['longitude'] = df_weather['location'].map(lambda loc: location_coords.get(loc, (None, None))[1])


Verify data

In [6]:
print(df_weather[['location', 'latitude', 'longitude']].head())


                 location   latitude  longitude
0  Aberdeen, South Dakota  45.464981 -98.487813
1  Aberdeen, South Dakota  45.464981 -98.487813
2  Aberdeen, South Dakota  45.464981 -98.487813
3  Aberdeen, South Dakota  45.464981 -98.487813
4  Aberdeen, South Dakota  45.464981 -98.487813


 get_nearest_station function, which calculates a bounding box ("extent") from the latitude and longitude.
applies that function to each row of your DataFrame using df.apply(), creating a new column ("noaa_station_id") with the resulting station ID.

In [8]:
import requests

NOAA_BASE_URL = "https://www.ncdc.noaa.gov/cdo-web/api/v2"

def get_nearest_station(lat, lon, radius_km=25):
    # Convert the radius in kilometers to an approximate degree offset
    delta = radius_km / 111  # Rough approximation (1 degree ≈ 111 km)
    extent = f"{lat - delta},{lon - delta},{lat + delta},{lon + delta}"
    
    headers = {"token": globals().get("NOAA_TOKEN", "bBLxgxcmJBuWGimWcdGXAndsRBvqoiYp")}
    params = {
        "limit": 1,
        "datasetid": "GHCND",
        "extent": extent,
    }
    response = requests.get(f"{NOAA_BASE_URL}/stations", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        results = data.get("results", [])
        if results:
            return results[0].get("id")
        else:
            print(f"No station found for lat={lat}, lon={lon}")
            return None
    else:
        print(f"NOAA API request failed (status {response.status_code}): {response.text}")
        return None

# Then, apply this function to your DataFrame row-wise:
df_weather['noaa_station_id'] = df_weather.apply(lambda row: get_nearest_station(row['latitude'], row['longitude']), axis=1)


NOAA API request failed (status 503): <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>

NOAA API request failed (status 503): <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>



KeyboardInterrupt: 

In [14]:
import pandas as pd

# Get the overall date range from your DataFrame
start_date = df_weather['datetime'].min().strftime('%Y-%m-%d')
end_date = df_weather['datetime'].max().strftime('%Y-%m-%d')
print("Date Range:", start_date, "to", end_date)


Date Range: 2020-12-31 to 2024-12-27


In [20]:
import requests

# Define the base URL for NOAA API
NOAA_BASE_URL = "https://www.ncdc.noaa.gov/cdo-web/api/v2"

def fetch_noaa_multiple(station_id, start_date, end_date, datatypes):
    """
    Fetch NOAA data for a given station over a date range for multiple datatypes.
    NOAA API accepts multiple datatypeid parameters by repeating the parameter.
    """
    headers = {"token": globals().get("NOAA_TOKEN", "bBLxgxcmJBuWGimWcdGXAndsRBvqoiYp")}
    # NOAA API allows repeating the parameter. Using a list comprehension to build parameters.
    params = {
        "datasetid": "GHCND",
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "limit": 1000
    }
    # Create a list of (key, value) pairs for datatypeid
    datatype_params = [("datatypeid", dt) for dt in datatypes]
    # Merge with the basic params. (requests accepts a list of tuples for parameters.)
    combined_params = list(params.items()) + datatype_params

    response = requests.get(f"{NOAA_BASE_URL}/data", headers=headers, params=combined_params)
    if response.status_code == 200:
        return response.json().get('results', [])
    else:
        print(f"Data request failed for station {station_id} (status {response.status_code}): {response.text}")
        return []

# Example datatypes you want:
desired_datatypes = ['PRCP', 'SNOW', 'TMAX']  # add more if available, e.g., 'SM' for soil moisture if it exists


In [ ]:
# Create a dictionary to store NOAA data for each station
station_data = {}

# Loop over each unique station in your DataFrame
for station_id in df['noaa_station_id'].dropna().unique():
    print(f"Fetching data for station {station_id}...")
    results = fetch_noaa_multiple(station_id, start_date, end_date, desired_datatypes)
    station_data[station_id] = results


In [21]:
# Use a known station and date range to test API response
test_station = station  # 'GHCND:ACW00011604'
test_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"  # Using the NOAA data endpoint

params = {
    "datasetid": "GHCND",
    "stationid": test_station,
    "startdate": start_date,
    "enddate": end_date,
    "limit": 5  # use a small limit for testing
}
headers = {"token": NOAA_TOKEN}

response = requests.get(test_url, headers=headers, params=params)
print("Status Code:", response.status_code)
print("Response Text:", response.text)

Status Code: 200
Response Text: {}
